In [5]:
import random as rd
import numpy as np
import pandas as pd
import os

from scoring import Metrics
from classification import Random

In [2]:
DIRPATH = 'src'

# Import data
import_path = os.path.join(DIRPATH, 'export.npy')

data = np.load (import_path)
#data[:, :-1] = data[:, :-1]/255.  # normalise values
vals_nb = data.shape[0]

In [3]:
# Create train and test sets

def create_sets(data, train_frac=0.05):

    # Create (train, test)
    data_shuffled = data.copy()

    np.random.seed(0)
    np.random.shuffle(data_shuffled)

    train_nb = int(vals_nb * train_frac)
    # print(f'training on {train_nb} vals')

    return data_shuffled[:train_nb], data_shuffled[train_nb:]

data_train, data_test = create_sets(data, train_frac=0.3)

In [10]:
from classifiers.lda import LDA

RUN_LDA = False
if RUN_LDA:
    data_train_lda = data_train
    data_test_lda  = data_test

    # Train LDA
    lda = LDA()
    lda.fit (data_train_lda[:, :-1], data_train_lda[:, -1])

    # Test LDA
    pred_lda = lda.eval_batch(data_test_lda[:, :-1], verbose=False)
    metrics_lda = Metrics(data_test_lda[:, -1], pred_lda)
    print(metrics_lda)

In [ ]:
from classifiers.qda import QDA

RUN_QDA = True
if RUN_QDA:
    data_train_qda = data_train
    data_test_qda  = data_test[:10000]

    # Train QDA
    qda = QDA()
    qda.fit (data_train_qda)

    # Test QDA
    pred_qda = qda.eval_batch(data_test_qda[:, :-1], verbose=False)
    metrics_qda = Metrics(data_test_qda[:, -1], pred_qda)
    print(metrics_qda)